# Get movement angle from joint posistion data & Labeling for NN training

## 1. Make a array from txt file
The file's path to be preprocessed must be entered directly. (file_path)

### Extract only digits
Extract only the joint coordinate values (digits) from a txt file and store them in an array

In [1]:
def Preprocessing (lines):
    output = []
    frame = []
    joint = 0
    for line in lines:
        joint+=1
        line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
        line = line.split(":")
        line = line[1]

        line = line.strip(')')
        line = line.strip('(')
        line = line.split(",")

        for i in range(3):
            line[i] = float(line[i])
        frame.append(line)

        if joint%32 == 0:
            output.append(frame)
            joint = 0
            frame = []
    return output

## 2. Get the Maximum angle
It is the function that finds the angle formed by the arm and the body. One angle is derived for each frame, and all angles for each frame derived in this way are stored in angleList.

In [2]:
import numpy as np

In [3]:
### 최대각도 구하기 #1 ###
#output[frame][joint][x,y,z]

# ∠
# the joints in test limb
def getAngleList(joints):
    angleList = []
    for i in range (len(joints)):
        joint_1_xyz = np.array(joints[i][4])  # a
        joint_2_xyz = np.array(joints[i][7])  # b
        # the joints in plane
        joint_3_xyz = np.array(joints[i][4])   # c
        joint_4_xyz = np.array(joints[i][18])   # d

        # the vectors of above three joints.
        vector1 = joint_2_xyz - joint_1_xyz
        vector2 = joint_4_xyz - joint_3_xyz     # Fig_11_2 m09 Active straight raise
        # Vector2 = (1, 0, 0)   # Fig_12_2 m11 Trunk stability

        # the norm of vectors
        norm_vector1 = np.sqrt(vector1.dot(vector1))
        norm_vector2 = np.sqrt(vector2.dot(vector2))
        # the dot product of vectors
        dot_product = vector1.dot(vector2)
        # the radians of the angle
        angle_cos = dot_product / (norm_vector1 * norm_vector2)
        # convert to an angle value
        angle_deg = (np.arccos(angle_cos) * 180) / np.pi

        angleList.append(angle_deg)
    return angleList

## 3. Make a Input for NN training
This is the function where the maximum value, minimum value, average, and variance of each element of angleList are obtained. When we make a model, they become input.

### Labeling 
This is the part for labeling. If the maximum angle of motion was in the range of 80-95 degrees, it was set as correct motion and labeled accordingly.</br>
Since we labeled correct motion as 0 and incorrect motion as 1, the if statement was written like that.

In [4]:
#get the data feature "x" for (max, min, mean, and variance)
import statistics as st

def calculate(angleList):
    X_max = max(angleList)
    X_min = min(angleList)
    X_mean = sum(angleList)/len(angleList)
    X_variance = st.pvariance(angleList)
    
    ### for Labeling ###
    if X_max > 80 and X_max <= 95:
        y = 0
    else:
        y = 1
    ####################
    
    return [X_max, X_min, X_mean, X_variance, int(y)]
#print(X_max,'\t', X_min ,'\t' ,X_mean ,'\t' ,X_variance)

## 4. Repeat steps 1-3 for all txt files
For all txt files in the ```txtfiles``` folder, the above 3 functions are executed and the input values to be used for the NN are stored as an array.

In [5]:
import os

path = './txtfiles/'
file_list = os.listdir(path)
files = [file for file in file_list if file.endswith('.txt')]
print(file_list)
features =[]

for file in files:
    with open(path+file, 'r') as f:
        lines = f.readlines()
        jointsCoordList = Preprocessing(lines)
        angleList = getAngleList(jointsCoordList)
        features.append(calculate(angleList))
        f.close()

['m12_1.txt', 'm12_2.txt', 'm12_2w.txt', 'm12_3.txt', 'm12_3w.txt', 'm12_4.txt', 'm12_4w.txt']


## 5. Make a .csv file
Record the maximum value, minimum value, mean, and variance obtained in step 3 in a csv file. At this time, the csv file must be in the same directory as the ``joint.data.extraction.ipynb`` file.

In [6]:
#write the data feature "x" for (max, min, mean, and variance) on the csv file
import csv

filename = "test.csv"

f = open(filename, "a", newline="")

writer = csv.writer(f)
for feat in features:
    writer.writerow(feat)
f.close()

In [7]:
import pandas as pd
csv_filepath = "C:/Users/soohyun/Downloads/jointData/"+filename
pd.read_csv(csv_filepath)

,max,min,mean,variance,y
0,89.252469,11.938456,49.161963,856.436477,0
1,93.129307,10.547509,48.159795,764.425423,0
2,80.885158,8.483955,44.820644,425.482344,0
3,85.948046,13.414578,37.542487,208.363189,0
4,72.144159,13.248240,36.401979,230.232890,1
5,99.083614,12.357153,49.703616,1043.003945,1
6,73.955996,12.177012,30.580823,346.531128,1
